In [1]:
import pandas as pd

df_operations = pd.read_excel(
    "k:/DOP/OED/METHOD&TOOLS/3 - PROJECTS/2 - ON GOING/2 - OE/2502 DIGITAL PROJECTS YEVGENIY/TRS/TRS_relations.xlsx",
    sheet_name='Operations')

In [2]:
df_operations = df_operations[['event', 'Category']]
mask = df_operations['Category'].notna()
df_operations_u = df_operations[mask].reset_index(drop=True)
df_operations_u

,event,Category
0,LG_PHASE_1,Planned_downtime
1,LG_PHASE_2,Planned_downtime
2,LG_PHASE_3,Planned_downtime
3,LG_PHASE_4_1,Planned_downtime
4,LG_PHASE_5_2,Planned_downtime
5,RP_BROKEN_CABLE,Unplanned_downtime_losses
6,RP_BROKEN_ELECTRICAL,Unplanned_downtime_losses
7,RP_BROKEN_GENERATOR,Unplanned_downtime_losses
8,RP_BROKEN_HYDRAULIC,Unplanned_downtime_losses
9,RP_BROKEN_PT_1200,Unplanned_downtime_losses


In [3]:
import pyodbc
print(pyodbc.drivers())

['SQL Server', 'SQL Server Native Client 11.0', 'ODBC Driver 17 for SQL Server']


In [4]:
driver = 'ODBC Driver 17 for SQL Server'
server = 'BKZTKDSDB41,21433'
database = 'PIFD'

# Строка подключения
conn_str = f'DRIVER={{{driver}}};SERVER={server};DATABASE={database};Trusted_Connection=yes;'

# SQL-запрос
query = """
SELECT 
    AFE.name COLLATE DATABASE_DEFAULT AS [equipment],
    AFET.name COLLATE DATABASE_DEFAULT AS [event],
    DATEADD(SECOND, (AVEvF.starttime / 10000000) - 63240134400 + 6 * 3600, '2005-01-01') AS starttime,
    IIF(AVEvF.endtime > 2638317188000000000, NULL, 
        DATEADD(SECOND, (AVEvF.endtime / 10000000) - 63240134400 + 6 * 3600, '2005-01-01')) AS endtime
FROM PIFD.dbo.AFEventFrame AS AVEvF
LEFT JOIN PIFD.dbo.AFElementTemplate AS AFET 
    ON AVEvF.fktemplateid = AFET.rid
LEFT JOIN PIFD.dbo.AFElement AS AFE 
    ON AVEvF.fkprimaryreferencedelement = AFE.rid
WHERE AFE.name COLLATE DATABASE_DEFAULT IN (
    SELECT DISTINCT equipment COLLATE DATABASE_DEFAULT 
    FROM PI_EXTRA.dbo.DR_REL_EQUIPMENT_EVENT
)
AND AFE.id IN (
    SELECT fkelementid 
    FROM PIFD.dbo.AFElementVersion 
    WHERE fktemplateid = (
        SELECT id 
        FROM PIFD.dbo.AFElementTemplate 
        WHERE name = 'RIG' 
        AND fkdatabaseid = (
            SELECT id FROM PIFD.dbo.AFDatabase WHERE name = 'DEM_DR'
        )
    )
)
"""

# Загрузка в DataFrame
try:
    with pyodbc.connect(conn_str) as conn:
        df_osi_operations = pd.read_sql(query, conn) # pyright: ignore[reportArgumentType]
        print(df_osi_operations.head())  # Проверка результата
except Exception as e:
    print("Ошибка при подключении или выполнении запроса:", e)

C:\Users\ykarabekov\AppData\Local\Temp\ipykernel_10340\3572333381.py:42: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_osi_operations = pd.read_sql(query, conn) # pyright: ignore[reportArgumentType]


       equipment         event           starttime             endtime
0  DBKAZ40 BL №1     RP_REPAIR 2021-08-20 12:00:00 2021-09-02 17:00:00
1  DBKAZ40 BL №1  CV_LONG_STOP 2020-04-27 08:00:00 2020-08-15 08:00:00
2  DBKAZ40 BL №1        WT_PPR 2020-02-18 08:00:00 2020-02-18 09:00:00
3  DBKAZ40 BL №1      WT_LUNCH 2020-02-18 12:00:00 2020-02-18 13:00:00
4  DBKAZ40 BL №1        WT_PPR 2020-02-18 20:00:00 2020-02-18 21:00:00


In [5]:
df_osi_operations['duration'] = df_osi_operations['endtime'] - df_osi_operations['starttime']

df_osi_operations['duration'] = df_osi_operations['duration'].dt.total_seconds() / 3600

df_osi_operations


,equipment,event,starttime,endtime,duration
0,DBKAZ40 BL №1,RP_REPAIR,2021-08-20 12:00:00,2021-09-02 17:00:00,317.0
1,DBKAZ40 BL №1,CV_LONG_STOP,2020-04-27 08:00:00,2020-08-15 08:00:00,2640.0
2,DBKAZ40 BL №1,WT_PPR,2020-02-18 08:00:00,2020-02-18 09:00:00,1.0
3,DBKAZ40 BL №1,WT_LUNCH,2020-02-18 12:00:00,2020-02-18 13:00:00,1.0
4,DBKAZ40 BL №1,WT_PPR,2020-02-18 20:00:00,2020-02-18 21:00:00,1.0
...,...,...,...,...,...
557494,ПБУ ЗМО-1500ПС №3,SB_ELECTRICAL_SHUTDOWN,2023-02-15 01:00:00,2023-02-16 08:00:00,31.0
557495,ПБУ ЗМО-1500ПС №3,RP_WAITING_FOR_SPARE_PART,2024-12-13 22:00:00,2024-12-15 09:00:00,35.0
557496,ПБУ ЗМО-1500ПС №3,SB_WAITING_FOR_WORK,2024-12-15 21:00:00,2024-12-17 21:00:00,48.0
557497,ПБУ ЗМО-1500ПС №3,RP_REPAIR,2024-12-17 21:00:00,2024-12-20 09:00:00,60.0


In [6]:
df_rigs = pd.read_excel(
    "k:/DOP/OED/METHOD&TOOLS/3 - PROJECTS/2 - ON GOING/2 - OE/2502 DIGITAL PROJECTS YEVGENIY/TRS/TRS_relations.xlsx",
    sheet_name='Rigs-Name')
df_rigs

,DrillCompany,Rig-AcQuire,Rig-osiDEM,Tipe of circulation
0,BurGeoProekt,PRAKLA_01-BGP,NaN,Direct
1,BurGeoProekt,PRAKLA_02-BGP,NaN,Direct
2,BurGeoProekt,PRAKLA_03-BGP,NaN,Direct
3,BurGeoProekt,PRAKLA_04-BGP,NaN,Direct
4,BurGeoProekt,ZIF1200_01-BGP,ZIF 1200 №1 Бургеопроект,Direct
...,...,...,...,...
157,VolkovGeology_#5,ZIF1200_36-E05,NaN,Direct
158,VolkovGeology_#5,ZIF1200_37-E05,NaN,Direct
159,VolkovGeology_#5,ZIF1200_38-E05,NaN,Direct
160,VolkovGeology_#5,ZIF1200_42-E05,NaN,Direct


In [7]:
df_circ_data = pd.read_excel(
    "k:/DOP/OED/METHOD&TOOLS/3 - PROJECTS/2 - ON GOING/2 - OE/2502 DIGITAL PROJECTS YEVGENIY/TRS/TRS_relations.xlsx",
    sheet_name='Standart avarage meters')
df_circ_data

,Circ,"Standard avarage drilling, m/h","time to well drill, h"
0,Direct,2.7,210
1,RC,5.4,87


In [8]:
df_osi_operations['endtime'].max()

Timestamp('2025-06-04 09:00:00')

In [9]:
database_a = 'acqkzdem1'
# SQL-запрос
query_2 = """
SELECT
    HOLEID,
    DrillCompany,
    DrillRig,
    HolePurpose,
    HoleStatus,
    ENDDATE,
    CASE 
        WHEN TRY_CAST([InvoiceDepth] AS FLOAT) IS NOT NULL AND TRY_CAST([InvoiceDepth] AS FLOAT) > 0 
            THEN CAST([InvoiceDepth] AS FLOAT)
        ELSE [DEPTH]
    END AS DEPTH
    FROM [ACQKZDEM1].[dbo].[AV_AREVA_V_DRILLING_SPREADSHEET_DOP]
    where [Sourse_DB]='DEM' 
      and NOT ([HoleStatus] = 'PROJECT' or [HoleStatus]='CONSTRUCTION')
      AND ENDDATE >= '2020-01-01'
"""

# Строка подключения
conn_str = f'DRIVER={{{driver}}};SERVER={server};DATABASE={database_a};Trusted_Connection=yes;'

# Загрузка данных в DataFrame
try:
    with pyodbc.connect(conn_str) as conn:
        df_meters = pd.read_sql(query_2, conn) # pyright: ignore[reportArgumentType]
        print(df_meters.head())  # Проверка
except Exception as e:
    print("Ошибка при подключении или выполнении запроса:", e)

C:\Users\ykarabekov\AppData\Local\Temp\ipykernel_10340\1890658307.py:28: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_meters = pd.read_sql(query_2, conn) # pyright: ignore[reportArgumentType]


           HOLEID  DrillCompany       DrillRig HolePurpose HoleStatus  \
0  MSK03_04_05_A_  BurGeoProekt  PRAKLA_04-BGP     PRODUCT     LIQUID   
1  MSK03_04_09_A_  BurGeoProekt  PRAKLA_04-BGP     PRODUCT     LIQUID   
2  MSK39_08_09_A4  BurGeoProekt  PRAKLA_01-BGP     PRODUCT   ACCEPTED   
3  MSK03_06_02_A_  BurGeoProekt  PRAKLA_03-BGP     PRODUCT     LIQUID   
4  MSK03_08_07_A_  BurGeoProekt  PRAKLA_03-BGP     PRODUCT   ACCEPTED   

     ENDDATE  DEPTH  
0 2020-09-16  503.0  
1 2020-09-17  506.0  
2 2020-04-05  508.0  
3 2020-04-13  475.0  
4 2020-09-26  507.0  


In [10]:
df_meters['HoleStatus'].value_counts()

HoleStatus
ACCEPTED          6485
LIQUID             481
NOT PROFITABLE     123
Name: count, dtype: int64

df_operations     - Название операций <br>
df_operations_u   - Название операций уникальные значения <br>
df_osi_operations - Время операций <br>
df_rigs           - Название агрегатов для связи <br>
df_circ_data      - Стандартные коэфициенты в зависим от циркуляции (скорость бурения в час, и время на одну скважину - час) <br>
df_meters         - Пробуренные метры

## Идея 
1 связать df_osi_operations - df_rigs (Взять за основу Rig-AcQuire не пустые)
2 связать df_osi_operations - df_df_operations_u


In [11]:
mask = df_rigs['Rig-AcQuire'].notna()
df_rigs = df_rigs[mask].reset_index(drop=True)
df_rigs.info()
df_osi_operations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162 entries, 0 to 161
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   DrillCompany         162 non-null    object
 1   Rig-AcQuire          162 non-null    object
 2   Rig-osiDEM           74 non-null     object
 3   Tipe of circulation  162 non-null    object
dtypes: object(4)
memory usage: 5.2+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 557499 entries, 0 to 557498
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   equipment  557499 non-null  object        
 1   event      557499 non-null  object        
 2   starttime  557499 non-null  datetime64[ns]
 3   endtime    557491 non-null  datetime64[ns]
 4   duration   557491 non-null  float64       
dtypes: datetime64[ns](2), float64(1), object(2)
memory usage: 21.3+ MB


In [12]:
df_osi_operations['equipment'] = df_osi_operations['equipment'].str.strip().str.lower()
df_rigs['Rig-osiDEM'] = df_rigs['Rig-osiDEM'].str.strip().str.lower()

df_merged = df_osi_operations.merge(
    df_rigs,
    left_on='equipment',
    right_on='Rig-osiDEM',
    how='inner'  # или 'inner', если нужны только совпадающие строки
)

df_merged

,equipment,event,starttime,endtime,duration,DrillCompany,Rig-AcQuire,Rig-osiDEM,Tipe of circulation
0,dbkaz40 bl №1,RP_REPAIR,2021-08-20 12:00:00,2021-09-02 17:00:00,317.0,KATCO,DBKAZ40-1,dbkaz40 bl №1,RC
1,dbkaz40 bl №1,CV_LONG_STOP,2020-04-27 08:00:00,2020-08-15 08:00:00,2640.0,KATCO,DBKAZ40-1,dbkaz40 bl №1,RC
2,dbkaz40 bl №1,WT_PPR,2020-02-18 08:00:00,2020-02-18 09:00:00,1.0,KATCO,DBKAZ40-1,dbkaz40 bl №1,RC
3,dbkaz40 bl №1,WT_LUNCH,2020-02-18 12:00:00,2020-02-18 13:00:00,1.0,KATCO,DBKAZ40-1,dbkaz40 bl №1,RC
4,dbkaz40 bl №1,WT_PPR,2020-02-18 20:00:00,2020-02-18 21:00:00,1.0,KATCO,DBKAZ40-1,dbkaz40 bl №1,RC
...,...,...,...,...,...,...,...,...,...
505968,пбу змо-1500пс №3,SB_ELECTRICAL_SHUTDOWN,2023-02-15 01:00:00,2023-02-16 08:00:00,31.0,KATCO,ZMO1500-3-KAT,пбу змо-1500пс №3,Direct
505969,пбу змо-1500пс №3,RP_WAITING_FOR_SPARE_PART,2024-12-13 22:00:00,2024-12-15 09:00:00,35.0,KATCO,ZMO1500-3-KAT,пбу змо-1500пс №3,Direct
505970,пбу змо-1500пс №3,SB_WAITING_FOR_WORK,2024-12-15 21:00:00,2024-12-17 21:00:00,48.0,KATCO,ZMO1500-3-KAT,пбу змо-1500пс №3,Direct
505971,пбу змо-1500пс №3,RP_REPAIR,2024-12-17 21:00:00,2024-12-20 09:00:00,60.0,KATCO,ZMO1500-3-KAT,пбу змо-1500пс №3,Direct


In [13]:
df_temp = df_merged.drop(columns=['equipment', 'Rig-osiDEM'])

In [14]:
df_operations_u

,event,Category
0,LG_PHASE_1,Planned_downtime
1,LG_PHASE_2,Planned_downtime
2,LG_PHASE_3,Planned_downtime
3,LG_PHASE_4_1,Planned_downtime
4,LG_PHASE_5_2,Planned_downtime
5,RP_BROKEN_CABLE,Unplanned_downtime_losses
6,RP_BROKEN_ELECTRICAL,Unplanned_downtime_losses
7,RP_BROKEN_GENERATOR,Unplanned_downtime_losses
8,RP_BROKEN_HYDRAULIC,Unplanned_downtime_losses
9,RP_BROKEN_PT_1200,Unplanned_downtime_losses


In [15]:
df_temp['event'] = df_temp['event'].str.strip().str.lower()
df_operations_u['event'] = df_operations_u['event'].str.strip().str.lower()

df_operations_total = df_temp.merge(df_operations_u, on='event', how='left')

df_operations_total['Category'] = df_operations_total['Category'].fillna('Speed_losess')

In [16]:
df_operations_total

,event,starttime,endtime,duration,DrillCompany,Rig-AcQuire,Tipe of circulation,Category
0,rp_repair,2021-08-20 12:00:00,2021-09-02 17:00:00,317.0,KATCO,DBKAZ40-1,RC,Unplanned_downtime_losses
1,cv_long_stop,2020-04-27 08:00:00,2020-08-15 08:00:00,2640.0,KATCO,DBKAZ40-1,RC,Speed_losess
2,wt_ppr,2020-02-18 08:00:00,2020-02-18 09:00:00,1.0,KATCO,DBKAZ40-1,RC,Planned_downtime
3,wt_lunch,2020-02-18 12:00:00,2020-02-18 13:00:00,1.0,KATCO,DBKAZ40-1,RC,Planned_downtime
4,wt_ppr,2020-02-18 20:00:00,2020-02-18 21:00:00,1.0,KATCO,DBKAZ40-1,RC,Planned_downtime
...,...,...,...,...,...,...,...,...
505968,sb_electrical_shutdown,2023-02-15 01:00:00,2023-02-16 08:00:00,31.0,KATCO,ZMO1500-3-KAT,Direct,Unplanned_downtime_losses
505969,rp_waiting_for_spare_part,2024-12-13 22:00:00,2024-12-15 09:00:00,35.0,KATCO,ZMO1500-3-KAT,Direct,Unplanned_downtime_losses
505970,sb_waiting_for_work,2024-12-15 21:00:00,2024-12-17 21:00:00,48.0,KATCO,ZMO1500-3-KAT,Direct,Unplanned_downtime_losses
505971,rp_repair,2024-12-17 21:00:00,2024-12-20 09:00:00,60.0,KATCO,ZMO1500-3-KAT,Direct,Unplanned_downtime_losses


In [17]:
# Убедимся, что endtime — datetime (хотя уже подтверждено)
df_operations_total['endtime'] = pd.to_datetime(df_operations_total['endtime'])
df_operations_total = df_operations_total.dropna(subset=['endtime'])

# Добавим год и месяц
df_operations_total['year'] = df_operations_total['endtime'].dt.year.astype('int16')
df_operations_total['month'] = df_operations_total['endtime'].dt.month.astype('int8')

pivot_df = df_operations_total.pivot_table(
    index=['DrillCompany', 'Rig-AcQuire', 'year', 'month', 'Tipe of circulation'],
    columns='Category',
    values='duration',
    aggfunc='sum',
    fill_value=0  # если нужно подставить 0 вместо NaN
).reset_index()

pivot_df = pivot_df.sort_values(
    by=['DrillCompany', 'Rig-AcQuire', 'year', 'month'],
    ascending=[True, True, True, True]
).reset_index(drop=True)


pivot_df


C:\Users\ykarabekov\AppData\Local\Temp\ipykernel_10340\1375976808.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_operations_total['year'] = df_operations_total['endtime'].dt.year.astype('int16')
C:\Users\ykarabekov\AppData\Local\Temp\ipykernel_10340\1375976808.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_operations_total['month'] = df_operations_total['endtime'].dt.month.astype('int8')


Category,DrillCompany,Rig-AcQuire,year,month,Tipe of circulation,Planned_downtime,Speed_losess,Unplanned_downtime_losses
0,BurGeoProekt,ZIF1200_01-BGP,2020,2,Direct,0.0,0.0,276.0
1,BurGeoProekt,ZIF1200_01-BGP,2020,3,Direct,47.0,420.0,278.0
2,BurGeoProekt,ZIF1200_01-BGP,2020,4,Direct,17.0,182.0,28.0
3,BurGeoProekt,ZIF1200_01-BGP,2021,1,Direct,9.0,7015.0,93.0
4,BurGeoProekt,ZIF1200_01-BGP,2021,2,Direct,84.0,397.0,191.0
...,...,...,...,...,...,...,...,...
2805,TechnoService-Eng,ZIF1200_16-TSE,2023,3,Direct,89.0,509.0,90.0
2806,TechnoService-Eng,ZIF1200_16-TSE,2023,4,Direct,76.0,468.0,106.0
2807,TechnoService-Eng,ZIF1200_16-TSE,2023,5,Direct,76.0,486.0,119.0
2808,TechnoService-Eng,ZIF1200_16-TSE,2023,6,Direct,77.0,405.0,133.0


In [18]:
import calendar

def hours_in_month(row):
    days = calendar.monthrange(row['year'], row['month'])[1]
    return days * 24

pivot_df['h_in_month'] = pivot_df.apply(hours_in_month, axis=1)

pivot_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2810 entries, 0 to 2809
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   DrillCompany               2810 non-null   object 
 1   Rig-AcQuire                2810 non-null   object 
 2   year                       2810 non-null   int16  
 3   month                      2810 non-null   int8   
 4   Tipe of circulation        2810 non-null   object 
 5   Planned_downtime           2810 non-null   float64
 6   Speed_losess               2810 non-null   float64
 7   Unplanned_downtime_losses  2810 non-null   float64
 8   h_in_month                 2810 non-null   int64  
dtypes: float64(3), int16(1), int64(1), int8(1), object(3)
memory usage: 162.0+ KB


In [19]:
# Преобразование ENDDATE в datetime формат
df_meters['ENDDATE'] = pd.to_datetime(df_meters['ENDDATE'], errors='coerce')

# Удаление строк с некорректными датами (NaT после преобразования)
df_meters.dropna(subset=['ENDDATE'], inplace=True)

df_meters


,HOLEID,DrillCompany,DrillRig,HolePurpose,HoleStatus,ENDDATE,DEPTH
0,MSK03_04_05_A_,BurGeoProekt,PRAKLA_04-BGP,PRODUCT,LIQUID,2020-09-16,503.0
1,MSK03_04_09_A_,BurGeoProekt,PRAKLA_04-BGP,PRODUCT,LIQUID,2020-09-17,506.0
2,MSK39_08_09_A4,BurGeoProekt,PRAKLA_01-BGP,PRODUCT,ACCEPTED,2020-04-05,508.0
3,MSK03_06_02_A_,BurGeoProekt,PRAKLA_03-BGP,PRODUCT,LIQUID,2020-04-13,475.0
4,MSK03_08_07_A_,BurGeoProekt,PRAKLA_03-BGP,PRODUCT,ACCEPTED,2020-09-26,507.0
...,...,...,...,...,...,...,...
7084,TNU39_08_05_A_,TechnoService-Eng,ZIF1200_01-TSE,PRODUCT,ACCEPTED,2025-05-11,297.0
7085,TNU39_08_03_B_,TechnoService-Eng,ZIF1200_08-TSE,PRODUCT,ACCEPTED,2025-05-22,298.0
7086,TNU39_08_07_A_,TechnoService-Eng,ZIF1200_04-TSE,PRODUCT,ACCEPTED,2025-05-26,296.0
7087,TNU39_13_09_A_,TechnoService-Eng,ZIF1200_03-TSE,INJECT,ACCEPTED,2025-05-26,292.0


In [20]:
df_meters['year'] = df_meters['ENDDATE'].dt.year.astype('int16')
df_meters['month'] = df_meters['ENDDATE'].dt.month.astype('int8')


pivot2 = pd.pivot_table(
    df_meters,
    index=['year', 'month', 'DrillRig'],
    columns='HoleStatus',
    values='HOLEID',
    aggfunc='count',
    fill_value=0
)

# Добавим суммарную глубину
depth_sum = df_meters.groupby(['year', 'month', 'DrillRig'])['DEPTH'].sum()

# Объединяем всё
pivot2['DEPTH'] = depth_sum

# Сбросим индекс, чтобы было удобно смотреть
meters_pivot = pivot2.reset_index()

meters_pivot

HoleStatus,year,month,DrillRig,ACCEPTED,LIQUID,NOT PROFITABLE,DEPTH
0,2020,1,DBKAZ40_1-KAT,7,0,0,2196.0
1,2020,1,PRAKLA_04-BGP,2,2,0,1069.0
2,2020,1,TWS685-1-KAT,3,0,0,925.0
3,2020,1,TWS685-2-KAT,1,0,0,308.0
4,2020,1,TWS685-3-KAT,2,0,0,621.0
...,...,...,...,...,...,...,...
2537,2025,6,ZIF1200_13-BGP,1,0,0,417.0
2538,2025,6,ZIF1200_14-TSE,1,0,0,282.0
2539,2025,6,ZIF1200_16-BGP,1,0,0,462.0
2540,2025,6,ZIF1200_25-BGP,1,0,0,418.0


In [21]:
pivot_df['Rig-AcQuire'] = pivot_df['Rig-AcQuire'].str.strip().str.upper()
meters_pivot['DrillRig'] = meters_pivot['DrillRig'].str.strip().str.upper()


total_merged_df = pd.merge(
    pivot_df,
    meters_pivot,
    left_on=['Rig-AcQuire', 'year', 'month'],
    right_on=['DrillRig', 'year', 'month'],
    how='left'  
)

total_merged_df.drop(columns='Rig-AcQuire', inplace=True)

total_merged_df


,DrillCompany,year,month,Tipe of circulation,Planned_downtime,Speed_losess,Unplanned_downtime_losses,h_in_month,DrillRig,ACCEPTED,LIQUID,NOT PROFITABLE,DEPTH
0,BurGeoProekt,2020,2,Direct,0.0,0.0,276.0,696,ZIF1200_01-BGP,0.0,1.0,0.0,80.0
1,BurGeoProekt,2020,3,Direct,47.0,420.0,278.0,744,ZIF1200_01-BGP,2.0,0.0,0.0,698.0
2,BurGeoProekt,2020,4,Direct,17.0,182.0,28.0,720,ZIF1200_01-BGP,2.0,0.0,0.0,697.0
3,BurGeoProekt,2021,1,Direct,9.0,7015.0,93.0,744,NaN,NaN,NaN,NaN,NaN
4,BurGeoProekt,2021,2,Direct,84.0,397.0,191.0,672,ZIF1200_01-BGP,2.0,0.0,0.0,998.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2805,TechnoService-Eng,2023,3,Direct,89.0,509.0,90.0,744,ZIF1200_16-TSE,2.0,0.0,0.0,587.0
2806,TechnoService-Eng,2023,4,Direct,76.0,468.0,106.0,720,ZIF1200_16-TSE,4.0,0.0,0.0,1132.0
2807,TechnoService-Eng,2023,5,Direct,76.0,486.0,119.0,744,ZIF1200_16-TSE,2.0,0.0,0.0,565.0
2808,TechnoService-Eng,2023,6,Direct,77.0,405.0,133.0,720,ZIF1200_16-TSE,3.0,0.0,0.0,1043.0


In [22]:
total_merged_df['Planned_production_time'] = total_merged_df['h_in_month'] - total_merged_df['Planned_downtime']

total_merged_df['Planned_factor'] = total_merged_df['Planned_production_time'] / total_merged_df['h_in_month']

# 1. Planned Production Time (PPT) с ограничением снизу
total_merged_df['Planned_production_time'] = (
    total_merged_df['h_in_month'] - total_merged_df['Planned_downtime']
).clip(lower=0)

# 2. Planned Factor 
total_merged_df['Planned_factor'] = (
    total_merged_df['Planned_production_time'] / total_merged_df['h_in_month']
).clip(lower=0)


total_merged_df.describe(include="all")


,DrillCompany,year,month,Tipe of circulation,Planned_downtime,Speed_losess,Unplanned_downtime_losses,h_in_month,DrillRig,ACCEPTED,LIQUID,NOT PROFITABLE,DEPTH,Planned_production_time,Planned_factor
count,2810,2810.000000,2810.000000,2810,2810.000000,2810.000000,2810.000000,2810.000000,2420,2420.000000,2420.000000,2420.000000,2420.000000,2810.000000,2810.000000
unique,5,NaN,NaN,2,NaN,NaN,NaN,NaN,66,NaN,NaN,NaN,NaN,NaN,NaN
top,BurGeoProekt,NaN,NaN,Direct,NaN,NaN,NaN,NaN,ZIF1200_29-BGP,NaN,NaN,NaN,NaN,NaN,NaN
freq,1057,NaN,NaN,2754,NaN,NaN,NaN,NaN,61,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,2022.602847,6.271886,NaN,81.613879,468.878826,142.168149,729.659786,NaN,2.546694,0.188430,0.047934,1193.421219,648.045907,0.888173
std,NaN,1.452313,3.505185,NaN,31.453063,247.292890,179.660293,20.158740,NaN,1.271971,0.434211,0.240950,525.010226,35.715516,0.042988
min,NaN,2020.000000,1.000000,NaN,0.000000,0.000000,0.000000,672.000000,NaN,0.000000,0.000000,0.000000,0.000000,379.000000,0.509409
25%,NaN,2021.000000,3.000000,NaN,69.000000,426.000000,72.000000,720.000000,NaN,2.000000,0.000000,0.000000,861.750000,630.000000,0.866667
50%,NaN,2023.000000,6.000000,NaN,85.000000,496.000000,106.000000,744.000000,NaN,2.000000,0.000000,0.000000,1074.500000,648.000000,0.883929
75%,NaN,2024.000000,9.000000,NaN,97.000000,541.000000,156.000000,744.000000,NaN,3.000000,0.000000,0.000000,1525.000000,667.000000,0.905556


In [23]:
# Gross Operating Time (GOT)
total_merged_df['Gross_operating_time'] = total_merged_df['Planned_production_time'] - total_merged_df['Unplanned_downtime_losses']

# Availability (доступность)
total_merged_df['Availability'] = total_merged_df['Gross_operating_time'] / total_merged_df['h_in_month']

# Gross Operating Time (GOT) с ограничением снизу
total_merged_df['Gross_operating_time'] = (
    total_merged_df['Planned_production_time'] - total_merged_df['Unplanned_downtime_losses']
).clip(lower=0)

# Availability (доступность) с ограничением снизу
total_merged_df['Availability'] = (
    total_merged_df['Gross_operating_time'] / total_merged_df['h_in_month']
).clip(lower=0)


total_merged_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2810 entries, 0 to 2809
Data columns (total 17 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   DrillCompany               2810 non-null   object 
 1   year                       2810 non-null   int16  
 2   month                      2810 non-null   int8   
 3   Tipe of circulation        2810 non-null   object 
 4   Planned_downtime           2810 non-null   float64
 5   Speed_losess               2810 non-null   float64
 6   Unplanned_downtime_losses  2810 non-null   float64
 7   h_in_month                 2810 non-null   int64  
 8   DrillRig                   2420 non-null   object 
 9   ACCEPTED                   2420 non-null   float64
 10  LIQUID                     2420 non-null   float64
 11  NOT PROFITABLE             2420 non-null   float64
 12  DEPTH                      2420 non-null   float64
 13  Planned_production_time    2810 non-null   float

In [24]:
# Назначаем коэффициенты по типу циркуляции
circulation_coeffs = df_circ_data.set_index('Circ')['Standard avarage drilling, m/h'].to_dict()

# Присваиваем коэффициенты каждому ряду
total_merged_df['circulation_coeff'] = total_merged_df['Tipe of circulation'].map(circulation_coeffs)

# Вычисляем потенциальную глубину бурения на основе Gross Operating Time
total_merged_df['Potential_depth'] = (
    total_merged_df['Gross_operating_time'] * total_merged_df['circulation_coeff']
).round(1)



total_merged_df.head(3)


,DrillCompany,year,month,Tipe of circulation,Planned_downtime,Speed_losess,Unplanned_downtime_losses,h_in_month,DrillRig,ACCEPTED,LIQUID,NOT PROFITABLE,DEPTH,Planned_production_time,Planned_factor,Gross_operating_time,Availability,circulation_coeff,Potential_depth
0,BurGeoProekt,2020,2,Direct,0.0,0.0,276.0,696,ZIF1200_01-BGP,0.0,1.0,0.0,80.0,696.0,1.000000,420.0,0.603448,2.7,1134.0
1,BurGeoProekt,2020,3,Direct,47.0,420.0,278.0,744,ZIF1200_01-BGP,2.0,0.0,0.0,698.0,697.0,0.936828,419.0,0.563172,2.7,1131.3
2,BurGeoProekt,2020,4,Direct,17.0,182.0,28.0,720,ZIF1200_01-BGP,2.0,0.0,0.0,697.0,703.0,0.976389,675.0,0.937500,2.7,1822.5


In [25]:
def calculate_net_operating_time(row):
    try:
        drilled = row['DEPTH']
        coeff = row['circulation_coeff']
        if pd.notnull(drilled) and pd.notnull(coeff) and coeff > 0:
            result = drilled / coeff
            return max(result, 0)  # отрицательные значения → 0
        else:
            return 0
    except:
        return 0

total_merged_df['Net_operating_time'] = total_merged_df.apply(calculate_net_operating_time, axis=1).round(1)


In [26]:
total_merged_df.head(2)

,DrillCompany,year,month,Tipe of circulation,Planned_downtime,Speed_losess,Unplanned_downtime_losses,h_in_month,DrillRig,ACCEPTED,LIQUID,NOT PROFITABLE,DEPTH,Planned_production_time,Planned_factor,Gross_operating_time,Availability,circulation_coeff,Potential_depth,Net_operating_time
0,BurGeoProekt,2020,2,Direct,0.0,0.0,276.0,696,ZIF1200_01-BGP,0.0,1.0,0.0,80.0,696.0,1.000000,420.0,0.603448,2.7,1134.0,29.6
1,BurGeoProekt,2020,3,Direct,47.0,420.0,278.0,744,ZIF1200_01-BGP,2.0,0.0,0.0,698.0,697.0,0.936828,419.0,0.563172,2.7,1131.3,258.5


In [27]:
def calculate_performance(row):
    try:
        net_time = row['Net_operating_time']
        gross_time = row['Gross_operating_time']
        if pd.notnull(net_time) and pd.notnull(gross_time) and gross_time > 0:
            result = net_time / gross_time
            return max(min(result, 1), 0)
        else:
            return 0
    except:
        return 0

total_merged_df['Performance'] = total_merged_df.apply(calculate_performance, axis=1)


total_merged_df.head(3)

,DrillCompany,year,month,Tipe of circulation,Planned_downtime,Speed_losess,Unplanned_downtime_losses,h_in_month,DrillRig,ACCEPTED,...,NOT PROFITABLE,DEPTH,Planned_production_time,Planned_factor,Gross_operating_time,Availability,circulation_coeff,Potential_depth,Net_operating_time,Performance
0,BurGeoProekt,2020,2,Direct,0.0,0.0,276.0,696,ZIF1200_01-BGP,0.0,...,0.0,80.0,696.0,1.000000,420.0,0.603448,2.7,1134.0,29.6,0.070476
1,BurGeoProekt,2020,3,Direct,47.0,420.0,278.0,744,ZIF1200_01-BGP,2.0,...,0.0,698.0,697.0,0.936828,419.0,0.563172,2.7,1131.3,258.5,0.616945
2,BurGeoProekt,2020,4,Direct,17.0,182.0,28.0,720,ZIF1200_01-BGP,2.0,...,0.0,697.0,703.0,0.976389,675.0,0.937500,2.7,1822.5,258.1,0.382370


In [28]:
# Назначаем коэффициенты по типу циркуляции
well_drill_time_coef = df_circ_data.set_index('Circ')['time to well drill, h'].to_dict()

# Присваиваем коэффициенты каждому ряду
total_merged_df['Well drill coef'] = total_merged_df['Tipe of circulation'].map(well_drill_time_coef)

# Вычисляем quality Gross Operating Time
total_merged_df['Valuable Operating Time'] = (
    total_merged_df['Net_operating_time'] - total_merged_df['Well drill coef'] * total_merged_df['LIQUID']
).round(1).clip(lower=0)

def calculate_quality(row):
    try:
        net_time = row['Valuable Operating Time']
        gross_time = row['Net_operating_time']
        if pd.notnull(net_time) and pd.notnull(gross_time) and gross_time > 0:
            result = net_time / gross_time
            return max(result, 0)  # убрать отрицательные
        else:
            return 0
    except:
        return 0

total_merged_df['quality'] = total_merged_df.apply(calculate_quality, axis=1)


total_merged_df['OEE'] = (
    total_merged_df['Availability'] *
    total_merged_df['Performance'] *
    total_merged_df['quality']
)


In [29]:
total_merged_df.head(3)

,DrillCompany,year,month,Tipe of circulation,Planned_downtime,Speed_losess,Unplanned_downtime_losses,h_in_month,DrillRig,ACCEPTED,...,Gross_operating_time,Availability,circulation_coeff,Potential_depth,Net_operating_time,Performance,Well drill coef,Valuable Operating Time,quality,OEE
0,BurGeoProekt,2020,2,Direct,0.0,0.0,276.0,696,ZIF1200_01-BGP,0.0,...,420.0,0.603448,2.7,1134.0,29.6,0.070476,210,0.0,0.0,0.000000
1,BurGeoProekt,2020,3,Direct,47.0,420.0,278.0,744,ZIF1200_01-BGP,2.0,...,419.0,0.563172,2.7,1131.3,258.5,0.616945,210,258.5,1.0,0.347446
2,BurGeoProekt,2020,4,Direct,17.0,182.0,28.0,720,ZIF1200_01-BGP,2.0,...,675.0,0.937500,2.7,1822.5,258.1,0.382370,210,258.1,1.0,0.358472


In [30]:
total_merged_df['TRS'] = (
    total_merged_df['OEE'] * total_merged_df['Planned_factor']
)

total_merged_df.head(3)


,DrillCompany,year,month,Tipe of circulation,Planned_downtime,Speed_losess,Unplanned_downtime_losses,h_in_month,DrillRig,ACCEPTED,...,Availability,circulation_coeff,Potential_depth,Net_operating_time,Performance,Well drill coef,Valuable Operating Time,quality,OEE,TRS
0,BurGeoProekt,2020,2,Direct,0.0,0.0,276.0,696,ZIF1200_01-BGP,0.0,...,0.603448,2.7,1134.0,29.6,0.070476,210,0.0,0.0,0.000000,0.000000
1,BurGeoProekt,2020,3,Direct,47.0,420.0,278.0,744,ZIF1200_01-BGP,2.0,...,0.563172,2.7,1131.3,258.5,0.616945,210,258.5,1.0,0.347446,0.325497
2,BurGeoProekt,2020,4,Direct,17.0,182.0,28.0,720,ZIF1200_01-BGP,2.0,...,0.937500,2.7,1822.5,258.1,0.382370,210,258.1,1.0,0.358472,0.350008


In [31]:
total_merged_df.columns

Index(['DrillCompany', 'year', 'month', 'Tipe of circulation',
       'Planned_downtime', 'Speed_losess', 'Unplanned_downtime_losses',
       'h_in_month', 'DrillRig', 'ACCEPTED', 'LIQUID', 'NOT PROFITABLE',
       'DEPTH', 'Planned_production_time', 'Planned_factor',
       'Gross_operating_time', 'Availability', 'circulation_coeff',
       'Potential_depth', 'Net_operating_time', 'Performance',
       'Well drill coef', 'Valuable Operating Time', 'quality', 'OEE', 'TRS'],
      dtype='object')

In [32]:
total_merged_df.columns = [
    "Drill Company", "Year", "Month", "Type Of Circulation", "Planned Downtime",
    "Speed Losses", "Unplanned Downtime Losses", "H In Month", "Drill Rig",
    "ACCEPTED", "LIQUID", "NOT PROFITABLE", "Total Drilled", "Planned Production Time",
    "Planned Factor", "Gross Operating Time", "Availability", "Circulation Coeff",
    "Potential Depth", "Net Operating Time", "Performance", "Well drill coef", 
    "Valuable Operating Time", "Quality", "OEE", "TRS"
]


total_merged_df.describe(include='all')


,Drill Company,Year,Month,Type Of Circulation,Planned Downtime,Speed Losses,Unplanned Downtime Losses,H In Month,Drill Rig,ACCEPTED,...,Availability,Circulation Coeff,Potential Depth,Net Operating Time,Performance,Well drill coef,Valuable Operating Time,Quality,OEE,TRS
count,2810,2810.000000,2810.000000,2810,2810.000000,2810.000000,2810.000000,2810.000000,2420,2420.000000,...,2810.000000,2810.000000,2810.000000,2810.000000,2810.000000,2810.000000,2420.000000,2810.000000,2810.000000,2810.000000
unique,5,NaN,NaN,2,NaN,NaN,NaN,NaN,66,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,BurGeoProekt,NaN,NaN,Direct,NaN,NaN,NaN,NaN,ZIF1200_29-BGP,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,1057,NaN,NaN,2754,NaN,NaN,NaN,NaN,61,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,2022.602847,6.271886,NaN,81.613879,468.878826,142.168149,729.659786,NaN,2.546694,...,0.704412,2.753808,1408.254520,377.093915,0.661260,207.548754,399.763595,0.780452,0.426626,0.373490
std,NaN,1.452313,3.505185,NaN,31.453063,247.292890,179.660293,20.158740,NaN,1.271971,...,0.141814,0.377408,315.188354,233.076005,0.348497,17.193016,200.397695,0.377781,0.260696,0.225896
min,NaN,2020.000000,1.000000,NaN,0.000000,0.000000,0.000000,672.000000,NaN,0.000000,...,0.000000,2.700000,0.000000,0.000000,0.000000,87.000000,0.000000,0.000000,0.000000,0.000000
25%,NaN,2021.000000,3.000000,NaN,69.000000,426.000000,72.000000,720.000000,NaN,2.000000,...,0.663978,2.700000,1313.600000,192.700000,0.421393,210.000000,228.500000,0.627841,0.232507,0.208193
50%,NaN,2023.000000,6.000000,NaN,85.000000,496.000000,106.000000,744.000000,NaN,2.000000,...,0.734459,2.700000,1447.200000,375.400000,0.738522,210.000000,375.900000,1.000000,0.470430,0.412315
75%,NaN,2024.000000,9.000000,NaN,97.000000,541.000000,156.000000,744.000000,NaN,3.000000,...,0.775000,2.700000,1541.700000,557.800000,1.000000,210.000000,555.300000,1.000000,0.655108,0.574253


In [33]:
total_merged_df['Speed Losses Calculated'] = (
    total_merged_df['Gross Operating Time'] - total_merged_df['Net Operating Time']
).clip(lower=0)  # Чтобы отрицательных не было

In [34]:
df_losses = pd.melt(
    total_merged_df,
    id_vars=[
        'Drill Company',
        'Drill Rig',
        'Year',
        'Month',
        'Type Of Circulation'
    ],
    value_vars=[
        'Planned Downtime',
        'Unplanned Downtime Losses',
        'Speed Losses Calculated',
        'Speed Losses'
    ],
    var_name='Loss Type',
    value_name='Loss Value'
)


In [35]:
df_productivity = pd.melt(
    total_merged_df,
    id_vars=[
        'Drill Company',
        'Drill Rig',
        'Year',
        'Month',
        'Type Of Circulation'
    ],
    value_vars=[
        'Planned Factor',
        'Availability',
        'Performance',
        'Quality',
        'OEE',
        'TRS'
    ],
    var_name='Productivity Type',
    value_name='Productivity Value'
)


df_productivity


,Drill Company,Drill Rig,Year,Month,Type Of Circulation,Productivity Type,Productivity Value
0,BurGeoProekt,ZIF1200_01-BGP,2020,2,Direct,Planned Factor,1.000000
1,BurGeoProekt,ZIF1200_01-BGP,2020,3,Direct,Planned Factor,0.936828
2,BurGeoProekt,ZIF1200_01-BGP,2020,4,Direct,Planned Factor,0.976389
3,BurGeoProekt,NaN,2021,1,Direct,Planned Factor,0.987903
4,BurGeoProekt,ZIF1200_01-BGP,2021,2,Direct,Planned Factor,0.875000
...,...,...,...,...,...,...,...
16855,TechnoService-Eng,ZIF1200_16-TSE,2023,3,Direct,TRS,0.257250
16856,TechnoService-Eng,ZIF1200_16-TSE,2023,4,Direct,TRS,0.520890
16857,TechnoService-Eng,ZIF1200_16-TSE,2023,5,Direct,TRS,0.252581
16858,TechnoService-Eng,ZIF1200_16-TSE,2023,6,Direct,TRS,0.479149


In [36]:
df_operations_total['Category'].value_counts()

Category
Speed_losess                 248113
Planned_downtime             183574
Unplanned_downtime_losses     74278
Name: count, dtype: int64

In [37]:
df_losses['Loss Type'].value_counts()

Loss Type
Planned Downtime             2810
Unplanned Downtime Losses    2810
Speed Losses Calculated      2810
Speed Losses                 2810
Name: count, dtype: int64

In [38]:
df_operations_total['Category'] = df_operations_total['Category'].replace({
    'Speed_losess': 'Speed Losses',
    'Planned_downtime': 'Planned Downtime',
    'Unplanned_downtime_losses': 'Unplanned Downtime Losses'
})

df_operations_total['Category'].value_counts()

C:\Users\ykarabekov\AppData\Local\Temp\ipykernel_10340\1908169926.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_operations_total['Category'] = df_operations_total['Category'].replace({


Category
Speed Losses                 248113
Planned Downtime             183574
Unplanned Downtime Losses     74278
Name: count, dtype: int64

In [39]:
df_operations_total.columns = [
    "Event",
    "Start Time",
    "End Time",
    "Duration",
    "Drill Company",
    "Rig",
    "Type Of Circulation",
    "Category",
    "Year",
    "Month"
]


In [40]:
df_operations_total['Event Category'] = df_operations_total['Event'].str.split('_').str[0].str.upper()
df_operations_total['Event'] = df_operations_total['Event'].str.split('_', n=1).str[1].str.capitalize()

df_events_duration = df_operations_total[
    [
        "Year",
        "Month",
        "Category",
        "Drill Company",
        "Rig",
        "Type Of Circulation",
        "Event Category",
        "Event",
        "Duration"
    ]
]
df_events_duration['Event'].value_counts()

C:\Users\ykarabekov\AppData\Local\Temp\ipykernel_10340\1191542425.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_operations_total['Event Category'] = df_operations_total['Event'].str.split('_').str[0].str.upper()
C:\Users\ykarabekov\AppData\Local\Temp\ipykernel_10340\1191542425.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_operations_total['Event'] = df_operations_total['Event'].str.split('_', n=1).str[1].str.capitalize()


Event
Ppr                   136547
Pilot_drilling         52840
Reaming                28807
Development            23642
Drill_rod_handling     19933
                       ...  
Swabb_locked               7
Air_lift                   5
Screen_destroy             5
Mud_pit_filling            4
Mantle_compressor          1
Name: count, Length: 107, dtype: int64

In [41]:
df_events_duration.sort_values(
    by=['Year', 'Month', 'Rig', 'Category', 'Duration'],
    ascending=[False, True, True, True, False],
    inplace=True
)

df_events_duration.head(3)

C:\Users\ykarabekov\AppData\Local\Temp\ipykernel_10340\2827919590.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_events_duration.sort_values(


,Year,Month,Category,Drill Company,Rig,Type Of Circulation,Event Category,Event,Duration
6591,2025,1,Planned Downtime,KATCO,PRAKLA_01-KAT,RC,WT,Ppr,1.0
6592,2025,1,Planned Downtime,KATCO,PRAKLA_01-KAT,RC,WT,Lunch,1.0
6594,2025,1,Planned Downtime,KATCO,PRAKLA_01-KAT,RC,WT,Ppr,1.0


In [42]:
df_meters.dropna(subset=['DrillCompany', 'DrillRig'], inplace=True)

df_meters.columns = [
    'Hole ID',
    'Drilling Company',
    'Drill Rig',
    'Purpose',
    'Status',
    'End_Date',
    'Depth_m',
    'Year',
    'Month'
]


In [43]:
import os

output_dir = 'k:/DOP/OED/METHOD&TOOLS/3 - PROJECTS/2 - ON GOING/2 - OE/2502 DIGITAL PROJECTS YEVGENIY/TRS/Python converted/'
os.makedirs(output_dir, exist_ok=True)

df_meters.to_csv(
    os.path.join(output_dir, 'df_meters.csv'),
    index=False,
    encoding='utf-8-sig'
)

df_events_duration.to_csv(
    os.path.join(output_dir, 'df_events_duration.csv'),
    index=False,
    encoding='utf-8-sig'
)



In [44]:
df_events_duration

,Year,Month,Category,Drill Company,Rig,Type Of Circulation,Event Category,Event,Duration
6591,2025,1,Planned Downtime,KATCO,PRAKLA_01-KAT,RC,WT,Ppr,1.0
6592,2025,1,Planned Downtime,KATCO,PRAKLA_01-KAT,RC,WT,Lunch,1.0
6594,2025,1,Planned Downtime,KATCO,PRAKLA_01-KAT,RC,WT,Ppr,1.0
6595,2025,1,Planned Downtime,KATCO,PRAKLA_01-KAT,RC,WT,Ppr,1.0
6597,2025,1,Planned Downtime,KATCO,PRAKLA_01-KAT,RC,WT,Lunch,1.0
...,...,...,...,...,...,...,...,...,...
279414,2020,12,Unplanned Downtime Losses,BurGeoProekt,ZIF1200_29-BGP,Direct,WT,Waiting_for_logging_result,1.0
279433,2020,12,Unplanned Downtime Losses,BurGeoProekt,ZIF1200_29-BGP,Direct,WT,Waiting_for_logging_result,1.0
279438,2020,12,Unplanned Downtime Losses,BurGeoProekt,ZIF1200_29-BGP,Direct,WT,Waiting_for_logging_result,1.0
279443,2020,12,Unplanned Downtime Losses,BurGeoProekt,ZIF1200_29-BGP,Direct,WT,Waiting_for_logging,1.0


In [45]:
df_losses.to_csv(
    os.path.join(output_dir, 'df_losses.csv'),
    index=False,
    encoding='utf-8-sig'
)

In [46]:
df_productivity.to_csv(
    os.path.join(output_dir, 'df_productivity.csv'),
    index=False,
    encoding='utf-8-sig'
)

In [47]:
dates = df_productivity[['Year', 'Month']].drop_duplicates().sort_values(['Year', 'Month']).reset_index(drop=True)


dates.to_csv(
    os.path.join(output_dir, 'df_dates.csv'),
    index=False,
    encoding='utf-8-sig'
)